In [1]:
import numpy as np
import math
from tqdm.notebook import tqdm
from functools import cache
import copy

import heapq



In [2]:
with open("input_day_19.txt") as f:
    text = f.read()


test = False
if test:
    text = r"""
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

test2 = False
if test2:
    text = """
qs{R}
px{a<2006:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,A}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

test3 = False
if test3:
    text = """
px{a<2006:A,R}
in{s<1351:px,A}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

test4 = False
if test4:
    text = """
in{s<1351:A,R}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

text = text.strip()


In [3]:
w_, p_ = text.split("\n\n")

w_dic = {}
for w in w_.split():
    name, rules = w[:-1].split("{")
    rules = rules.split(",")
    w_dic[name] = rules

parts = []
for p in p_.split():
    strs = p[1:-1].split(",")
    nums = [int(x.split("=")[1]) for x in strs]
    parts.append(nums)

In [4]:
total = 0
for p in parts:
    #print("analyzing", p)
    x, m, a, s = p
    curr_rules = w_dic["in"]
    curr_rules_name = "in"
    done_with_part = False
    while True:
        if done_with_part:
            break
        #print("  new set of rules", curr_rules_name, curr_rules)
        for r in curr_rules:
            #print("    rule", r)
            if done_with_part:
                continue
            if r == "R":
                dest = "R"
                t = True
            elif r == "A":
                dest = "A"
                t = True
            elif r[1] == ">":
                dest = r.split(":")[1]
                num = int(r.split('>')[1].split(":")[0])
                if r[0] == "x": t = x > num
                if r[0] == "m": t = m > num
                if r[0] == "a": t = a > num
                if r[0] == "s": t = s > num
            elif r[1] == "<":
                dest = r.split(":")[1]
                num = int(r.split('<')[1].split(":")[0])
                if r[0] == "x": t = x < num
                if r[0] == "m": t = m < num
                if r[0] == "a": t = a < num
                if r[0] == "s": t = s < num
            else:
                dest = r
                t = True
            
            if t:
                if dest == "A":
                    total += x + m + a + s
                    done_with_part = True
                    #print("DONE WITH PART")
                elif dest == "R":
                    done_with_part = True
                else:
                    new_rules = w_dic[dest]
                    new_rules_name = dest
                    break
        if done_with_part:
            break
        curr_rules = new_rules
        curr_rules_name = new_rules_name

        
print("part 1:", total)



part 1: 409898


In [5]:
total = 0

def handle_rng(rng, workflow_name, rule_index, print_prefix=""):
    global total

    r = w_dic[workflow_name][rule_index]
    print(print_prefix + "handling range:", rng, workflow_name, rule_index, r)

    if r == "R":
        return
    elif r == "A":
        #print("adding", rng, (rng[0][1] - rng[0][0]), (rng[1][1] - rng[1][0]), (rng[2][1] - rng[2][0]), (rng[3][1] - rng[3][0]))
        total += (rng[0][1] - rng[0][0]) * (rng[1][1] - rng[1][0]) * (rng[2][1] - rng[2][0]) * (rng[3][1] - rng[3][0])
        return
    elif not (("<" in r) or (">" in r)):
        handle_rng(rng, r, 0)
        return
    else:
        if r[0] == "x": att_index = 0
        elif r[0] == "m": att_index = 1
        elif r[0] == "a": att_index = 2
        elif r[0] == "s": att_index = 3
        else:
            assert False, "bad! rule: " + r

        #print("  rule:", r)

        if r[1] == ">":
            dest = r.split(":")[1]
            num = int(r.split('>')[1].split(":")[0]) + 1 # this is the number that the entire range has to be greater than or equal to
            if rng[att_index][0] < num < rng[att_index][1]: # need to split the ranges
                print(print_prefix + "  splitting")
                new_lower_rng = (rng[att_index][0], num)
                new_upper_rng = (num, rng[att_index][1])

                lower_rng = copy.deepcopy(rng)
                lower_rng[att_index] = new_lower_rng
                upper_rng = copy.deepcopy(rng)
                upper_rng[att_index] = new_upper_rng

                handle_rng(lower_rng, workflow_name, rule_index, print_prefix=print_prefix + "  ")
                handle_rng(upper_rng, workflow_name, rule_index, print_prefix=print_prefix + "  ")
                return

            elif num <= rng[att_index][0]: # entire range included
                if dest == "A":
                    print(print_prefix + "  accepted")
                    #print("adding", rng, (rng[0][1] - rng[0][0]), (rng[1][1] - rng[1][0]), (rng[2][1] - rng[2][0]), (rng[3][1] - rng[3][0]))
                    total += (rng[0][1] - rng[0][0]) * (rng[1][1] - rng[1][0]) * (rng[2][1] - rng[2][0]) * (rng[3][1] - rng[3][0])
                    return
                if dest == "R":
                    print(print_prefix + "  rejected")
                    return
                
                print(print_prefix + "  moving to dest")
                handle_rng(rng, dest, 0, print_prefix=print_prefix + "  ")
                return
            
            elif rng[att_index][1] <= num: # entire range excluded
                print(print_prefix + "  moving to next rule")
                handle_rng(rng, workflow_name, rule_index + 1, print_prefix=print_prefix + "  ")
                return
            
            else:
                assert False, "bad 2!"

        elif r[1] == "<":
            #print("  in < part")
            dest = r.split(":")[1]
            num = int(r.split('<')[1].split(":")[0]) # this is the number that the entire range has to be less than
            #print("  range must be less than", num)
            #print(rng, att_index)
            if rng[att_index][0] < num < rng[att_index][1]: # need to split the ranges
                print(print_prefix + "  splitting")
                #print("  splitting")
                new_lower_rng = (rng[att_index][0], num)
                new_upper_rng = (num, rng[att_index][1])

                lower_rng = copy.deepcopy(rng)
                lower_rng[att_index] = new_lower_rng
                upper_rng = copy.deepcopy(rng)
                upper_rng[att_index] = new_upper_rng

                handle_rng(lower_rng, workflow_name, rule_index, print_prefix=print_prefix + "  ")
                handle_rng(upper_rng, workflow_name, rule_index, print_prefix=print_prefix + "  ")
                return

            elif rng[att_index][1] <= num: # entire range included

                if dest == "A":
                    print(print_prefix + "  accepted")
                    #print("adding", rng, (rng[0][1] - rng[0][0]), (rng[1][1] - rng[1][0]), (rng[2][1] - rng[2][0]), (rng[3][1] - rng[3][0]))
                    total += (rng[0][1] - rng[0][0]) * (rng[1][1] - rng[1][0]) * (rng[2][1] - rng[2][0]) * (rng[3][1] - rng[3][0])
                    return
                if dest == "R":
                    print(print_prefix + "  rejected")
                    return

                #print("  entire included")
                print(print_prefix + "  moving to dest")
                handle_rng(rng, dest, 0, print_prefix=print_prefix + "  ")
                return
            
            elif num <= rng[att_index][0]: # entire range excluded
                print(print_prefix + "  moving to next rule")
                #print("  entire excluded")
                handle_rng(rng, workflow_name, rule_index + 1, print_prefix=print_prefix + "  ")
                return
            
            else:
                assert False, "bad 2!"

handle_rng([(1, 4001), (1, 4001), (1, 4001), (1, 4001)], "in", 0)
#handle_rng([(2663, 4001), (1, 4001), (1, 2006), (1, 1351)], "in", 0)


print("fast part 2:", total)
        

handling range: [(1, 4001), (1, 4001), (1, 4001), (1, 4001)] in 0 a<1847:rc
  splitting
  handling range: [(1, 4001), (1, 4001), (1, 1847), (1, 4001)] in 0 a<1847:rc
    moving to dest
    handling range: [(1, 4001), (1, 4001), (1, 1847), (1, 4001)] rc 0 s>2109:xcj
      splitting
      handling range: [(1, 4001), (1, 4001), (1, 1847), (1, 2110)] rc 0 s>2109:xcj
        moving to next rule
        handling range: [(1, 4001), (1, 4001), (1, 1847), (1, 2110)] rc 1 m>2310:kh
          splitting
          handling range: [(1, 4001), (1, 2311), (1, 1847), (1, 2110)] rc 1 m>2310:kh
            moving to next rule
            handling range: [(1, 4001), (1, 2311), (1, 1847), (1, 2110)] rc 2 xkp
handling range: [(1, 4001), (1, 2311), (1, 1847), (1, 2110)] xkp 0 m<986:hr
  splitting
  handling range: [(1, 4001), (1, 986), (1, 1847), (1, 2110)] xkp 0 m<986:hr
    moving to dest
    handling range: [(1, 4001), (1, 986), (1, 1847), (1, 2110)] hr 0 m<591:ztv
      splitting
      handling range: [(

In [6]:
x_key_vals = [0.5]
m_key_vals = [0.5]
a_key_vals = [0.5]
s_key_vals = [0.5]

for name, rules in w_dic.items():
    #print(rules)
    for r in rules:
        if ">" in r:
            num = int(r.split('>')[1].split(":")[0])
            if r[0] == "x": x_key_vals.append(num + 0.5)
            if r[0] == "m": m_key_vals.append(num + 0.5)
            if r[0] == "a": a_key_vals.append(num + 0.5)
            if r[0] == "s": s_key_vals.append(num + 0.5)
        elif "<" in r:
            num = int(r.split('<')[1].split(":")[0])
            if r[0] == "x": x_key_vals.append(num - 0.5)
            if r[0] == "m": m_key_vals.append(num - 0.5)
            if r[0] == "a": a_key_vals.append(num - 0.5)
            if r[0] == "s": s_key_vals.append(num - 0.5)

x_key_vals.append(4000.5)
m_key_vals.append(4000.5)
a_key_vals.append(4000.5)
s_key_vals.append(4000.5)

x_key_vals = list(set(x_key_vals))
m_key_vals = list(set(m_key_vals))
a_key_vals = list(set(a_key_vals))
s_key_vals = list(set(s_key_vals))

x_key_vals.sort()
m_key_vals.sort()
a_key_vals.sort()
s_key_vals.sort()

x_test_vals = [int(x + 0.5) for x in x_key_vals[:-1]]
m_test_vals = [int(m + 0.5) for m in m_key_vals[:-1]]
a_test_vals = [int(a + 0.5) for a in a_key_vals[:-1]]
s_test_vals = [int(s + 0.5) for s in s_key_vals[:-1]]

x_widths = [int(x_key_vals[i+1] - x_key_vals[i]) for i in range(len(x_key_vals) - 1)]
m_widths = [int(m_key_vals[i+1] - m_key_vals[i]) for i in range(len(m_key_vals) - 1)]
a_widths = [int(a_key_vals[i+1] - a_key_vals[i]) for i in range(len(a_key_vals) - 1)]
s_widths = [int(s_key_vals[i+1] - s_key_vals[i]) for i in range(len(s_key_vals) - 1)]

#print(x_key_vals)
#print(x_test_vals)
#print(x_widths)

print("")
print(len(x_test_vals), len(m_test_vals), len(a_test_vals), len(s_test_vals))
print(len(x_test_vals) * len(m_test_vals) * len(a_test_vals) * len(s_test_vals))


total = 0
with tqdm(total=len(x_test_vals) * len(m_test_vals) * len(a_test_vals) * len(s_test_vals)) as pbar:
    for x_i, x in enumerate(x_test_vals):
        for m_i, m in enumerate(m_test_vals):
            for a_i, a in enumerate(a_test_vals):
                for s_i, s in enumerate(s_test_vals):
                    curr_rules = w_dic["in"]
                    curr_rules_name = "in"
                    done_with_part = False
                    while True:
                        if done_with_part:
                            break
                        #print("  new set of rules", curr_rules_name, curr_rules)
                        for r in curr_rules:
                            #print("    rule", r)
                            if done_with_part:
                                continue
                            if r == "R":
                                dest = "R"
                                t = True
                            elif r == "A":
                                dest = "A"
                                t = True
                            elif r[1] == ">":
                                dest = r.split(":")[1]
                                num = int(r.split('>')[1].split(":")[0])
                                if r[0] == "x": t = x > num
                                if r[0] == "m": t = m > num
                                if r[0] == "a": t = a > num
                                if r[0] == "s": t = s > num
                            elif r[1] == "<":
                                dest = r.split(":")[1]
                                num = int(r.split('<')[1].split(":")[0])
                                if r[0] == "x": t = x < num
                                if r[0] == "m": t = m < num
                                if r[0] == "a": t = a < num
                                if r[0] == "s": t = s < num
                            else:
                                dest = r
                                t = True
                            
                            if t:
                                if dest == "A":
                                    total += x_widths[x_i] * m_widths[m_i] * a_widths[a_i] * s_widths[s_i]
                                    done_with_part = True
                                    #print("DONE WITH PART")
                                elif dest == "R":
                                    done_with_part = True
                                else:
                                    new_rules = w_dic[dest]
                                    new_rules_name = dest
                                    break
                        if done_with_part:
                            break
                        curr_rules = new_rules
                        curr_rules_name = new_rules_name

                    pbar.update(1)
        
print("slow part 2:", total)



259 224 216 286
3583996416


  0%|          | 0/3583996416 [00:00<?, ?it/s]

KeyboardInterrupt: 